In [121]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [122]:
df=pd.read_csv("/kaggle/input/amazon-stock-price-all-time/Amazon.csv")

In [123]:
df.head()

In [124]:
df.tail()

In [125]:
df.shape

In [126]:
df.info()

In [127]:
df.columns

In [128]:
df.isnull().sum()

In [129]:
df=df.drop('Adj Close',axis=1)

In [130]:
df.head(2)

In [131]:
import matplotlib.pyplot as plt

In [132]:
fig,ax=plt.subplots(figsize=(20,8))
ax.plot(df['Date'],df['Close'],c='b')
ax.xaxis.set_major_locator(plt.MaxNLocator(15))
ax.set_xlabel('Date',fontsize='15')
ax.set_ylabel('Price in USD',fontsize='15')
plt.title("Amazon Stock Prices",fontsize='15')
plt.grid()
plt.show()

In [133]:
fig,ax=plt.subplots(figsize=(20,8))
ax.bar(df['Date'],df['Volume'])
ax.xaxis.set_major_locator(plt.MaxNLocator(15))
ax.set_xlabel('Date',fontsize='15')
ax.set_ylabel('Volumes',fontsize='15')
plt.title("Volumes Trends",fontsize='15')
plt.grid()
plt.show()

In [134]:
df['Market Cap']=df['Open']*df['Volume']

In [135]:
df.head(2)

In [136]:
fig,ax=plt.subplots(figsize=(20,8))
ax.plot(df['Date'],df['Market Cap'],c='r')
ax.xaxis.set_major_locator(plt.MaxNLocator(15))
ax.set_xlabel('Date',fontsize='15')
ax.set_ylabel('Market Cap',fontsize='15')
plt.title("Market Capita",fontsize='15')
plt.grid()
plt.show()

In [137]:
df.iloc[df['Market Cap'].argmax()]

In [157]:
import warnings 
warnings.filterwarnings('ignore')
import seaborn as sns
from datetime import datetime

In [138]:
df['Vol']=(df['Close']/df['Close'].shift(1))-1

In [145]:
df.head(2)

In [140]:
fig,ax=plt.subplots(figsize=(20,8))
ax.plot(df['Date'],df['Vol'],c='purple')
ax.xaxis.set_major_locator(plt.MaxNLocator(15))
plt.title("Volatility",fontsize='15')
plt.grid()
plt.show()

In [141]:
df['Vol'].hist(bins=100,color='blue')

In [142]:
df['Cummulative Return']=(1+df['Vol']).cumprod()

In [143]:
df.head(2)

In [144]:
fig,ax=plt.subplots(figsize=(20,8))
ax.plot(df['Date'],df['Cummulative Return'],c='y')
ax.xaxis.set_major_locator(plt.MaxNLocator(15))
ax.set_xlabel('Date',fontsize='15')
ax.set_ylabel('Cummulative Return',fontsize='15')
plt.title("Cummulative Return",fontsize='15')
plt.grid()
plt.show()

In [161]:
df.iloc[df['Cummulative Return'].argmax()]

In [162]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
import math

In [163]:
df['Date']=pd.to_datetime(df['Date'])
df.set_index('Date',inplace=True)

In [164]:
data=df.filter(['Close'])
dataset=data.values
training_data_len=math.ceil(len(dataset)*.8)
training_data_len

In [165]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
scaled_data

In [167]:
train_data=scaled_data[0:training_data_len,:]
x_train=[]
y_train=[]
for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
    if i<=60:
        print(x_train)
        print(y_train)
        print()

In [168]:
x_train,y_train=np.array(x_train),np.array(y_train)

In [170]:
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [171]:
model=Sequential()
model.add(LSTM(64,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(64,return_sequences=False))
model.add(Dense(32))
model.add(Dense(1))

In [172]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [175]:
model.fit(x_train,y_train,batch_size=1,epochs=10)

In [176]:
test_data=scaled_data[training_data_len-60:, :]
x_test=[]
y_test=dataset[training_data_len:,:]
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])

In [177]:
x_test=np.array(x_test)

In [178]:
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
x_test.shape

In [179]:
prediction=model.predict(x_test)
prediction=scaler.inverse_transform(prediction)

In [181]:
rmse=np.sqrt(np.mean(prediction - y_test)**2)
rmse

In [185]:
train=data[:training_data_len]
valid=data[training_data_len:]
valid['Prediction']=prediction
plt.figure(figsize=(16,8))
plt.title("Model")
plt.xlabel("Date",fontsize=18)
plt.ylabel("Close Price",fontsize=18)
plt.plot(train['Close'],linewidth=3.5)
plt.plot(valid[['Close','Prediction']],linewidth=3.5)
plt.legend(['Train','Valid','Prediction'],loc='upper center')

In [186]:
valid